### Regras


In [68]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [69]:
def analise_estoque(filial):
    df_saldo_estoque = pd.read_excel(f'planilhas/estoque/saldo_estoque_{filial}.xlsx', header=11, usecols=('B,C,F,G,H,I,J,O,P,Q,S,U,X'))[0:-3]
    df_saldo_estoque = df_saldo_estoque.set_axis(['ean','produto','laboratorio','grupo','curva','estoque_minimo','demanda','estoque','preco_custo','preco_venda','lucro','preco_custo_total','preco_venda_total'], axis=1)
    df_saldo_estoque['grupo'] = df_saldo_estoque['grupo'].astype(int)
    df_saldo_estoque['preco_custo_total'] = df_saldo_estoque['preco_custo_total'].str.replace('.', '').str.replace(',', '.')
    df_saldo_estoque['preco_custo'] = df_saldo_estoque['preco_custo'].str.replace('.', '').str.replace(',', '.')
    df_saldo_estoque['estoque_minimo'] = pd.to_numeric(df_saldo_estoque['estoque_minimo'], errors='coerce')
    df_saldo_estoque['preco_custo_total'] = pd.to_numeric(df_saldo_estoque['preco_custo_total'], errors='coerce')
    df_saldo_estoque['preco_custo'] = pd.to_numeric(df_saldo_estoque['preco_custo'], errors='coerce')
    df_saldo_estoque['ean'] = df_saldo_estoque['ean'].fillna(0).astype(int)

    df_saldo_estoque_filtrado = df_saldo_estoque.query('estoque_minimo > 0')
    valor_em_estoque = df_saldo_estoque['preco_custo_total'].sum()
    df_faltas = df_saldo_estoque_filtrado.query('estoque == 0')
    df_faltas['valor_faltas'] = df_faltas['estoque_minimo']*df_faltas['preco_custo']
    valor_faltas = df_faltas['valor_faltas'].sum()
    
    return valor_em_estoque, valor_faltas, df_saldo_estoque

def analise_estoque_grupo(df_saldo_estoque_grupo, grupo):
    df_saldo_estoque_grupo = df_saldo_estoque_grupo.query(f'grupo == {grupo}')
    df_saldo_estoque_grupo_filtrado = df_saldo_estoque_grupo.query('estoque_minimo > 0')
    valor_em_estoque = df_saldo_estoque_grupo['preco_custo_total'].sum()
    df_faltas = df_saldo_estoque_grupo_filtrado
    df_faltas['valor_faltas'] = df_faltas['estoque_minimo']*df_faltas['preco_custo']
    df_faltas['ean'] = df_faltas['ean'].astype(str)
    valor_faltas = df_faltas['valor_faltas'].sum()
    
    df_excesso = df_saldo_estoque_grupo.query('estoque > 0')
    df_excesso['excesso'] = np.where(df_excesso['estoque']>df_excesso['demanda'],df_excesso['estoque']-df_excesso['demanda'],0)
    df_excesso = df_excesso.query('excesso > 0')
    df_excesso['ean'] = df_excesso['ean'].astype(str)
    return valor_em_estoque, valor_faltas, df_faltas, df_excesso

##### somar demandas de Centrinho, Matriz e Canasvieiras

In [70]:
valor_em_estoque_matriz, valor_faltas_matriz, df_estoque_matriz = analise_estoque('001')

In [71]:
valor_em_estoque_centrinho, valor_faltas_centrinho, df_estoque_centrinho = analise_estoque('004')

In [72]:
valor_em_estoque_canasvieiras, valor_faltas_canasvieiras, df_estoque_canasvieiras = analise_estoque('009')

In [73]:
#filtrando perfumaria
df_estoque_matriz = df_estoque_matriz.query('grupo == 9000')
df_estoque_centrinho = df_estoque_centrinho.query('grupo == 9000')
df_estoque_canasvieiras = df_estoque_canasvieiras.query('grupo == 9000')

In [74]:
df_estoque_matriz = df_estoque_matriz[['ean','produto','estoque_minimo','demanda']]
df_estoque_matriz

,ean,produto,estoque_minimo,demanda
4,-2147483648,!BOBINA CUPOM FISCAL 8CMX40M,0,0.0
6,-2147483648,!COLETOR PERF DESCARBOX 13L,0,0.0
7,-2147483648,!COLETOR PERFUR DESCARPACK 13L,0,0.0
42,0,*TESTE GRAV CLEARBLUE CAN 1UN,0,0.0
143,-2147483648,ABS POS PARTO ULTRAFRAL C/18 U,1,1.0
...,...,...,...,...
21352,-2147483648,UNHA DE GATO 500MG 60CAP,0,0.0
21511,-2147483648,UXI COM UNHA DE GATO 60 CAPS 5,0,0.0
21542,-2147483648,VALERIANA,0,0.0
21598,-2147483648,VASELINA POMADA 20G,0,0.0


In [75]:
df_estoque_centrinho = df_estoque_centrinho[['ean','produto','estoque_minimo','demanda']]
df_estoque_centrinho

,ean,produto,estoque_minimo,demanda
4,-2147483648,!BOBINA CUPOM FISCAL 8CMX40M,0,0.0
6,-2147483648,!COLETOR PERF DESCARBOX 13L,0,0.0
7,-2147483648,!COLETOR PERFUR DESCARPACK 13L,0,0.0
42,0,*TESTE GRAV CLEARBLUE CAN 1UN,0,0.0
143,-2147483648,ABS POS PARTO ULTRAFRAL C/18 U,1,1.0
...,...,...,...,...
21352,-2147483648,UNHA DE GATO 500MG 60CAP,0,0.0
21511,-2147483648,UXI COM UNHA DE GATO 60 CAPS 5,0,0.0
21542,-2147483648,VALERIANA,0,0.0
21598,-2147483648,VASELINA POMADA 20G,0,0.0


In [76]:
df_estoque_canasvieiras = df_estoque_canasvieiras[['ean','produto','estoque_minimo','demanda']]
df_estoque_canasvieiras

,ean,produto,estoque_minimo,demanda
4,-2147483648,!BOBINA CUPOM FISCAL 8CMX40M,0,0.0
6,-2147483648,!COLETOR PERF DESCARBOX 13L,0,0.0
7,-2147483648,!COLETOR PERFUR DESCARPACK 13L,0,0.0
42,0,*TESTE GRAV CLEARBLUE CAN 1UN,0,0.0
143,-2147483648,ABS POS PARTO ULTRAFRAL C/18 U,1,1.0
...,...,...,...,...
21352,-2147483648,UNHA DE GATO 500MG 60CAP,0,0.0
21511,-2147483648,UXI COM UNHA DE GATO 60 CAPS 5,0,0.0
21542,-2147483648,VALERIANA,0,0.0
21598,-2147483648,VASELINA POMADA 20G,0,0.0


In [77]:
df_geral = pd.merge(df_estoque_matriz,df_estoque_centrinho, on=['ean','produto'], how='outer')
df_geral

,ean,produto,estoque_minimo_x,demanda_x,estoque_minimo_y,demanda_y
0,-2147483648,!BOBINA CUPOM FISCAL 8CMX40M,0,0.0,0,0.0
1,-2147483648,!COLETOR PERF DESCARBOX 13L,0,0.0,0,0.0
2,-2147483648,!COLETOR PERFUR DESCARPACK 13L,0,0.0,0,0.0
3,0,*TESTE GRAV CLEARBLUE CAN 1UN,0,0.0,0,0.0
4,-2147483648,ABS POS PARTO ULTRAFRAL C/18 U,1,1.0,1,1.0
...,...,...,...,...,...,...
1070,-2147483648,UNHA DE GATO 500MG 60CAP,0,0.0,0,0.0
1071,-2147483648,UXI COM UNHA DE GATO 60 CAPS 5,0,0.0,0,0.0
1072,-2147483648,VALERIANA,0,0.0,0,0.0
1073,-2147483648,VASELINA POMADA 20G,0,0.0,0,0.0


In [78]:
df_geral['estoque_minimo'] = df_geral['estoque_minimo_x'] + df_geral['estoque_minimo_y']
df_geral['demanda'] = df_geral['demanda_x'] + df_geral['demanda_y']
df_geral = df_geral[['ean','produto','estoque_minimo','demanda']]
df_geral

,ean,produto,estoque_minimo,demanda
0,-2147483648,!BOBINA CUPOM FISCAL 8CMX40M,0,0.0
1,-2147483648,!COLETOR PERF DESCARBOX 13L,0,0.0
2,-2147483648,!COLETOR PERFUR DESCARPACK 13L,0,0.0
3,0,*TESTE GRAV CLEARBLUE CAN 1UN,0,0.0
4,-2147483648,ABS POS PARTO ULTRAFRAL C/18 U,2,2.0
...,...,...,...,...
1070,-2147483648,UNHA DE GATO 500MG 60CAP,0,0.0
1071,-2147483648,UXI COM UNHA DE GATO 60 CAPS 5,0,0.0
1072,-2147483648,VALERIANA,0,0.0
1073,-2147483648,VASELINA POMADA 20G,0,0.0


In [79]:
df_estoque_canasvieiras

,ean,produto,estoque_minimo,demanda
4,-2147483648,!BOBINA CUPOM FISCAL 8CMX40M,0,0.0
6,-2147483648,!COLETOR PERF DESCARBOX 13L,0,0.0
7,-2147483648,!COLETOR PERFUR DESCARPACK 13L,0,0.0
42,0,*TESTE GRAV CLEARBLUE CAN 1UN,0,0.0
143,-2147483648,ABS POS PARTO ULTRAFRAL C/18 U,1,1.0
...,...,...,...,...
21352,-2147483648,UNHA DE GATO 500MG 60CAP,0,0.0
21511,-2147483648,UXI COM UNHA DE GATO 60 CAPS 5,0,0.0
21542,-2147483648,VALERIANA,0,0.0
21598,-2147483648,VASELINA POMADA 20G,0,0.0


In [80]:
df_geral = pd.merge(df_geral,df_estoque_canasvieiras, on=['ean','produto'], how='outer')

In [81]:
df_geral['estoque_minimo'] = df_geral['estoque_minimo_x'] + df_geral['estoque_minimo_y']
df_geral['demanda'] = df_geral['demanda_x'] + df_geral['demanda_y']
df_geral = df_geral[['ean','produto','estoque_minimo','demanda']]
df_geral

,ean,produto,estoque_minimo,demanda
0,-2147483648,!BOBINA CUPOM FISCAL 8CMX40M,0,0.0
1,-2147483648,!COLETOR PERF DESCARBOX 13L,0,0.0
2,-2147483648,!COLETOR PERFUR DESCARPACK 13L,0,0.0
3,0,*TESTE GRAV CLEARBLUE CAN 1UN,0,0.0
4,-2147483648,ABS POS PARTO ULTRAFRAL C/18 U,3,3.0
...,...,...,...,...
1188,-2147483648,UNHA DE GATO 500MG 60CAP,0,0.0
1189,-2147483648,UXI COM UNHA DE GATO 60 CAPS 5,0,0.0
1190,-2147483648,VALERIANA,0,0.0
1191,-2147483648,VASELINA POMADA 20G,0,0.0


In [82]:
df_geral.query('estoque_minimo > 0')

,ean,produto,estoque_minimo,demanda
4,-2147483648,ABS POS PARTO ULTRAFRAL C/18 U,3,3.0
5,-2147483648,ACALENTUS MELATONINA 90CAP,2,2.0
8,-2147483648,ADVANSEPT 70% SPR 30ML,1,1.0
9,-2147483648,AGUA BORICADA 3% LIQ 100ML,3,3.0
11,-2147483648,AGUA OXIG 10V AVVIO SPR 100ML,2,2.0
...,...,...,...,...
1171,-2147483648,TORNOZELEIRA HIDROL EL PR M 1U,2,2.0
1173,-2147483648,TORNOZELEIRA HIDROL EL PR PP 1,1,1.0
1174,-2147483648,TORNOZELEIRA HIDROL LON G 1UN,2,2.0
1180,-2147483648,TORNOZELEIRA MOVA LON PR G 1UN,1,1.0


In [83]:
filename = 'planilhas/estoque/demanda_trindade_9000.xlsx'
#df_geral.to_excel(filename)